# Investment Case Group Project

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
inputDir = 'C:/Users/ankit.bhatia/Documents/GitHub/PythonScripts/data/spark_funds/'
outDir = 'C:/Users/ankit.bhatia/Documents/GitHub/PythonScripts/data/spark_funds/outDir/'
file1 = 'companies.txt'
file2 = 'rounds2.csv'
file3 = 'mapping.csv'

## Checkpoint 1: Data Cleaning

In [ ]:
# Loading datasets into pandas dataframe.
companies = pd.read_csv(inputDir+file1,sep='\t',encoding='iso-8859-1')
rounds2 = pd.read_csv(inputDir+file2,encoding='iso-8859-1') # Use 'iso-8859-1' for accented characters
rounds2.info()

In [ ]:
# Get some insight
companies.head(2)

In [ ]:
rounds2.head(2)

### Table 1.1
#### Q1. How many unique companies are present in rounds2?

In [ ]:
# .count() will give the total no of rows.
#rounds2.company_permalink.count()

# Unique count which excludes NA as well by default
rounds2.company_permalink.nunique()

#### Q2. How many unique companies are present in companies?

In [ ]:
companies.permalink.nunique()

#### Q3. In the companies data frame, which column can be used as the unique key for each company? Write the name of the column.

In [ ]:
# Ist Method
#nunique() will give unique values for each column, column with maximum unique counts can be considered as unique column#### 
companies.nunique().sort_values(ascending=False).iloc[0:1]

# IInd Method
# Companies.describe() will give count, unique, top etc information, we store count & unique values in a temp df and
# then subtracting unique values(1) from count(0) and giving the column name where difference is 0
#companies.describe().iloc[0:2]


#### Q4. Are there any companies in the rounds2 file which are not  present in companies ?

In [ ]:
# Check the existacce of a column of dataframe1 in column1 of dataframe2
rounds2[rounds2['company_permalink'].str.lower().isin(companies['permalink'].str.lower())== False]

#### Q5. Merge the two data frames so that all  variables (columns)  in the companies frame are added to the rounds2 data frame. Name the merged frame master_frame. How many observations are present in master_frame ?

In [ ]:
# lower() the key columns of both dataframes, so they can match in case of diffence in case.
companies['permalink'] = companies['permalink'].str.lower()
rounds2['company_permalink'] = rounds2['company_permalink'].str.lower()
master_frame = pd.merge(left = companies,right = rounds2, how ='inner', left_on='permalink',right_on='company_permalink')
master_frame.shape[0]
# From here we will work with master_frame only

In [ ]:
# Drop Rows from master_frame where raised_amount_usd is null
master_frame = master_frame.drop(master_frame[master_frame.raised_amount_usd.isnull()].index)

#### Define Function

In [ ]:
# Defining a method to wite(.csv) a file to the user location.
def writeDF(dataframe,filename,dir_path):
    dataframe.to_csv(path_or_buf = outDir+filename+'.csv')
    size = str(round((os.path.getsize(outDir+filename+'.csv')/1000),1))
    print('File: '+filename+'.csv created at '+outDir+ '\nSize of file :'+size+ ' KB' )

#### Storing Data

In [ ]:
# Storing master_frame dataframe at user location, we can then use this .csv directly in Tableau
writeDF(master_frame,'master_frame',outDir)

## Checkpoint 2: Funding Type Analysis
### Table 2.1 ( Average Values of Investments for Each of these Funding Types)

#### Define Function

In [ ]:
# defining a function to get average funding amount for any funding type.
def avg_funding_amount(funding_type):
    return master_frame[master_frame['funding_round_type']==funding_type]['raised_amount_usd'].mean()

#### Q1/2/3/4. Average funding amount of different funding type ?

In [ ]:
venture = avg_funding_amount('venture')
angel = avg_funding_amount('angel')
seed = avg_funding_amount('seed')
private_equity = avg_funding_amount('private_equity')

print('venture' ,avg_funding_amount('venture'))
print('angel' ,avg_funding_amount('angel'))
print('seed' ,avg_funding_amount('seed'))
print('private_equity' ,avg_funding_amount('private_equity'))

#### Q5. Considering that Spark Funds wants to invest between 5 to 15 million USD per  investment round, which investment type is the most suitable for them?

In [ ]:
# We can check from the last step that only venture funding type lies between 5 to 15 million.
# Alternatively we can find the funding type through below code:
## [['']] double brackets or (.reset_index()) are used to keep the dataframe , if we use [''], it will be converted into series
most_suitable = master_frame.groupby(by='funding_round_type')[['raised_amount_usd']].mean()
most_suitable = most_suitable[(most_suitable.raised_amount_usd >= 5000000) & (most_suitable.raised_amount_usd <= 15000000)]
most_suitable

In [ ]:
# Filter the master_frame dataset based on the above conditions(funding_round_type)
master_frame_filtered = master_frame[master_frame.funding_round_type == most_suitable.index[0]]

master_frame_filtered.head(3)
# Now master_frame_filtered contains the data where funding_round_type ='venture' & raised_amount_usd between 5 to 15 million.

## Checkpoint 3: Country Analysis
### Table 3.1 ( Analysing the Top 3 English-Speaking Countries)

In [ ]:
top9 = master_frame_filtered.groupby('country_code')['raised_amount_usd'].sum().sort_values(ascending=False).reset_index()[0:9]
top9.count()

In [ ]:
# Now the requirement is to fetch top 3 countries from top 9 dataframe where official language is English, we will fetch this 
# information from MySQl > world database > country & country language table. Alternatively we can check this information
# from the pdf provided, but there we have to hardcode the information.
# Note : To match the countries in the countrylanguage table data with pdf data, I have inserted one entry of IND with
# official language is English.(update COUNTRYLANGUAGE set ISOFFICIAL='T' where countrycode= 'IND')
# installed mysql connector for python using conda install -c anaconda mysql-connector-python

import mysql.connector as con
db_connection = con.connect(host='localhost', database='world', user='root', password='mysql@123')
query = 'select code,name from country where code in(SELECT countrycode FROM COUNTRYLANGUAGE WHERE LANGUAGE=%s AND ISOFFICIAL=%s)'

#countryEN contains all the country codes where official language is English
countryEN =  pd.read_sql(sql =query,con =db_connection,params=['English','T'])

In [ ]:
# Add a new column identifier, IsOfficialEN in top9 dataframe.
top9['IsOfficialEN'] = top9.country_code.isin(countryEN['code'])
top9

#### Storing Data

In [ ]:
# Storing top9 dataframe at user location, we can then use this .csv directly in Tableau
writeDF(top9,'top9',outDir)

#### Q1/2/3. Top/Second/Third English speaking country ?

In [ ]:
top3 = top9[top9.IsOfficialEN].iloc[0:3,0:2].reset_index()
top = top3.country_code[0]
second = top3.country_code[1]
third = top3.country_code[2]

print('top ',top)
print('second ',second)
print('third ',third)

In [ ]:
# Filter master_frame_filtered dataset based on top 3 english speaking countries
top3_data = master_frame_filtered.loc[master_frame_filtered.country_code.isin(top3.country_code)]
top3_data.head(3)

## Checkpoint 4: Sector Analysis 1
### Table 5.1 ( Sector-wise Investment Analysis)

In [ ]:
# first string before the | will be considered the primary sector.
# Add the new column in master_frame_filtered dataframe named as 'primary_sector'
top3_data['primary_sector'] = top3_data['category_list'].str.split('|').str.get(0)
top3_data.head(3)

In [ ]:
mapping =  pd.read_csv(inputDir+file3,encoding='iso-8859-1') # Use 'iso-8859-1' for accented characters
mapping.head(3)

#### Define Function

In [ ]:
# Defining function demap, which will convert data in mapping dataframe above to two columns output.
def demap(df,outColName):
    l = list()
    c = int(df.count().sort_values(ascending=False)[0])
    for i in range(c):
        for col_name in df.columns:
            if df[col_name][i]==1:
               l.append(col_name)
    df2 = df.copy()
    df2[outColName] = l
    return df2.iloc[:,[0,-1]]

In [ ]:
mapping_new = demap(mapping,'main_sector')
mapping_new.head()

In [ ]:
top3_data_sector_wise = pd.merge(left=top3_data, right=mapping_new,left_on='primary_sector',right_on='category_list')
# Drop unnecessary columns
top3_data_sector_wise.drop(labels=['category_list_x', 'category_list_y'],axis=1, inplace=True)

In [ ]:
top3_data_sector_wise.head(3)

#### Storing Data

In [ ]:
# Storing master_frame_sector_wise dataframe at user location, we can then use this .csv directly in Tableau
# master_frame_sector_wise :: Contains the data for top3 english speaking countries where a particular funding type falling 
#within the 5-15 million USD range.

writeDF(top3_data_sector_wise,'top3_data_sector_wise',outDir)

## Checkpoint 5: Sector Analysis 2

Created three separate data frames D1, D2 and D3 for each of the three english countries containing the observations of funding type 'Venture' falling within the 5-15 million USD range.

In [ ]:
#USA
D1 = top3_data_sector_wise[top3_data_sector_wise['country_code']==top]

#GBR
D2 = top3_data_sector_wise[top3_data_sector_wise['country_code']==second]

#IND
D3 = top3_data_sector_wise[top3_data_sector_wise['country_code']==third]

D3.head(3)

#### Q1. Total number of Investments (count)

In [ ]:
print(D1.count()[0])
print(D2.count()[0])
print(D3.count()[0])

 #### Q2. Total amount of investment (USD)

In [ ]:
print(D1.raised_amount_usd.sum())
print(D2.raised_amount_usd.sum())
print(D3.raised_amount_usd.sum())

In [ ]:
# Define function to get top sectors count and name
def getSector(dataset,n,identifier):
    # dataset     :: datframe
    # n           :: n represents nth order, 0 for top, 1 for second top and so on.
    # identifier  :: identifier = 'count' to get the count, identifier = 'name' to get the sector name.
    if identifier == 'name':
        return (dataset.groupby(by='main_sector')['main_sector'].count().sort_values(ascending=False).index[n])
    elif identifier == 'count':
        return (dataset.groupby(by='main_sector')['main_sector'].count().sort_values(ascending=False)[n])

#### Q3. Top sector (based on count of investments)

In [ ]:
D1_sec0_name = getSector(dataset=D1,n=0,identifier='name')
D2_sec0_name = getSector(D2,0,'name')
D3_sec0_name = getSector(D3,0,'name')

print('D1_sec0_name : ',D1_sec0_name,'\nD2_sec0_name : ',D2_sec0_name,'\nD3_sec0_name : ',D3_sec0_name)

#### Q4. Second-best sector (based on count of investments)

In [ ]:
D1_sec1_name = getSector(D1,1,'name')
D2_sec1_name = getSector(D2,1,'name')
D3_sec1_name = getSector(D3,1,'name')

print('D1_sec1_name : ',D1_sec1_name,'\nD2_sec1_name : ',D2_sec1_name,'\nD3_sec1_name : ',D3_sec1_name)

#### Q5. Third-best sector (based on count of investments)

In [ ]:
D1_sec2_name = getSector(D1,2,'name')
D2_sec2_name = getSector(D2,2,'name')
D3_sec2_name = getSector(D3,2,'name')

print('D1_sec2_name : ',D1_sec2_name,'\nD2_sec2_name : ',D2_sec2_name,'\nD3_sec2_name : ',D3_sec2_name)

#### Q6. Number of investments in the top sector (refer to point 3)

In [ ]:
D1_sec0_count = getSector(D1,0,'count')
D2_sec0_count = getSector(D2,0,'count')
D3_sec0_count = getSector(D3,0,'count')

print('D1_sec0_count : ',D1_sec0_count,'\nD2_sec0_count : ',D2_sec0_count,'\nD3_sec0_count : ',D3_sec0_count)

#### Q7. Number of investments in the second-best sector (refer to point 4)

In [ ]:
D1_sec1_count = getSector(D1,1,'count')
D2_sec1_count = getSector(D2,1,'count')
D3_sec1_count = getSector(D3,1,'count')

print('D1_sec1_count : ',D1_sec1_count,'\nD2_sec1_count : ',D2_sec1_count,'\nD3_sec1_count : ',D3_sec1_count)

#### Q8. Number of investments in the third-best sector (refer to point 5)

In [ ]:
D1_sec2_count = getSector(D1,2,'count')
D2_sec2_count = getSector(D2,2,'count')
D3_sec2_count = getSector(D3,2,'count')

print('D1_sec2_count : ',D1_sec2_count,'\nD2_sec2_count : ',D2_sec2_count,'\nD3_sec2_count : ',D3_sec2_count)

#### Q9. For the top sector count-wise (point 3), which company received the highest investment?

In [ ]:
# Filter dataframes by top-sector names(Q3), then group by company name and sum the raised_amount_usd, then fetch highest value
D1_com0_invt = D1[D1.main_sector == D1_sec0_name].groupby(by='name')['raised_amount_usd'].sum().sort_values(ascending=False).index[0]
D2_com0_invt = D2[D2.main_sector == D2_sec0_name].groupby(by='name')['raised_amount_usd'].sum().sort_values(ascending=False).index[0]
D3_com0_invt = D3[D3.main_sector == D3_sec0_name].groupby(by='name')['raised_amount_usd'].sum().sort_values(ascending=False).index[0]

print('D1_com0_invt : ',D1_com0_invt,'\nD2_com0_invt : ',D2_com0_invt,'\nD3_com0_invt : ',D3_com0_invt)

#### Q10. For the second-best sector count-wise (point 4), which company received the highest investment?

In [ ]:
D1_com1_invt = D1[D1.main_sector == D1_sec1_name].groupby(by='name')['raised_amount_usd'].sum().sort_values(ascending=False).index[1]
D2_com1_invt = D2[D2.main_sector == D2_sec1_name].groupby(by='name')['raised_amount_usd'].sum().sort_values(ascending=False).index[1]
D3_com1_invt = D3[D3.main_sector == D3_sec1_name].groupby(by='name')['raised_amount_usd'].sum().sort_values(ascending=False).index[1]

print('D1_com1_invt : ',D1_com1_invt,'\nD2_com1_invt : ',D2_com1_invt,'\nD3_com1_invt : ',D3_com1_invt)

## Checkpoint 6: Plots

#### Q1. A plot showing the fraction of total investments (globally) in venture, seed, and private equity, and the average amount of investment in each funding type. This chart should make it clear that a certain funding type (FT) is best suited for Spark Funds.

#### Using Matplotlib

In [ ]:
selected_funding_round_type = ['venture','angel','seed','private_equity']
plot_frame = master_frame.loc[master_frame.funding_round_type.isin(selected_funding_round_type)]
plot_frame = plot_frame.groupby('funding_round_type')['raised_amount_usd'].mean().sort_values(ascending=True).reset_index()
cond = ((plot_frame.raised_amount_usd >= 5000000) & (plot_frame.raised_amount_usd <= 15000000))

x = range(len(plot_frame.funding_round_type))
y = plot_frame.raised_amount_usd/ 1000000

plt.figure(figsize=(9,6))

# Bar plot
plt.bar(x,y,color=cond.map({True: 'g', False: 'k'}))
# Color the funding round type with different color where raised amount used is between 5 to 15 million.

# Labels
plt.xlabel('Funding Round Type')
plt.ylabel('Raised Amount Used in Million(s)')

# Ticks
plt.xticks(x, plot_frame.funding_round_type)

# Horizontal Line
plt.axhline(y=5, color='b', linestyle='-',alpha = 0.5)
plt.text(max(x)+.7, 5,'5 Million')
plt.axhline(y=15, color='b', linestyle='-',alpha = 0.5)
plt.text(max(x)+.7, 15,'15 Million')

# Save as image
plt.savefig(outDir+'Funding_Type.png',bbox_inches="tight")

# Show plot
plt.show()